# Computing firing rates of ganglion cells

This IPython notebook is written by [Sahar Pirmoradian](https://www.linkedin.com/in/spirmorad/) as part of the European [RENVISION](http://www.renvision-fp7.eu) project for analyzing neural responses to full-field stimulus (a sequence of light and dark flashes), while the cells are recorded using a high-density multielectrode arrays (4096 electrodes). 

Version 1.1, 19 February 2015

## Importing necessary libraries

In [1]:
import h5py
import numpy as np
import scipy.io
import os.path
from scipy import stats
import matplotlib as mplt
import matplotlib.pyplot as plt
import pylab
from sklearn.neighbors.kde import KernelDensity
from scipy.stats import poisson

## Reading and preprocessing data

### Setting data and stimulus file names

In [2]:
masterdr = '/Users/sahar/Documents/spktanlz/data/retina/P38_06Mar14/ret2'

# data file is an hdf5 file (spikes, channels, ...)
datafullfn = masterdr + '/hdf5/Retina02_LeftEye_Stim02_ClassStimSimp_SpkD45_v18_clusterIDs_0.3_1_combined_merged.hdf5'

# stimulus file is a mat file that contains SamplingFrequency, frameSamples, frameTimes, pixels_allframes (see read_stimfile())  
stimfullfn = masterdr + '/mat/Retina02_LeftEye_FF_triggerf_stim.mat'

### Setting parameters 

In [3]:
figpath = masterdr + '/figs/' # where firing rate figures are stored
frparams_matfile = figpath + 'frparams' # it will be stored as a mat file
spksrc = 'cl'  # spike source of our data, 'ch' (channels), 'cl' (clusters), 'su' (sorted units)
coord_spksrc = 'rc' # coordination of channles on the chip, 'rc' (row-column) or 'cr' (column-row) 
minamp = 0 # minimum amplitude of spikes
minspk = 30 # minimum spikes a cell must have in Full-field experiment to be considered in analyses; if the number of FF trials are 30, we assume a responding cell fires at least once to each trial, that is minimum spikes are 30
no_stim_pertrial = 2 # black and light are presented in the full-field
slctdchs = np.array([6,7])# 'all' # if you like to analyze the firing rate of all cells, set slctdchs = 'all'; otherwise if you analzye only the fr of selected cells, e.g. 13 or 752, set slctdchs = np.array([13,752]); for a range of cells e.g. from Cell 10 to Cell 20, sltdchs=np.arange(10,20)
figformat = '.png'
dt =.04 # .05 #.025 #  binning
kernel = 'gaussian' #'exponential'
maxlatency =1000
minlatency = 0 #0#
maxduration = 1000
baseline_dur = .25 # int(starttime_exp) #stim['stimframes'][0] #5 sec
checkRatePeakZFLG = True # only units with considerably high firing rate are considered
max_poissonval_acceptable = .005 #1e-03
convwithspktrainFLG = True # False # is convolution with PSTH
firststim = 'l' # the first stimulus in the full-field is light ('l') or dark ('d') 

frplotFLG = 1
latencyplotFLG = 0
durplotFLG = 1
biasidxFLG = 0
d2lcmpFLG = 0 #dark 2 light comparison

frsaveparamsFLG = True

### Reading data file

In [20]:
def read_hdf5(fullfn, spksrc='ch'):
  # fullfn - string; full file name of an hdf5 file to be read
  # spksrc - 'ch' (channels), 'cl' (clusters), or 'su' (sorted units); spike source
  # OUTPUT:
  # spikes - array; frame number at which spikes were generated
  # chs    - array; channels or clusters that generated corresponding spikes 
  # amps   - array; the amplitude of spikes
  # freq   - float; sampling frequency. The frequency at which neuron signals were recorded
  # nbins  - int; the resolution of channel locations; it's usually 64.
  # loc2d  - array2D; the 2d location of channels on a chip   
  # loc    - array1D; the location of channels that has been converted to 1 dimension for simplicity
  # locspks2d - array2D; the location of spikes (not channels)

  try:  
    myfile = h5py.File(fullfn,'r')
  except:
    raise IOError('\nI cannot open your hdf5 file: %s' % fullfn)

  spikes = np.array(myfile['Times'].value, dtype=int)
  amps = myfile['Amplitudes'].value
  freq = myfile['Sampling'].value
  loc2d, locspks2d = (np.array([]), np.array([]))
  if (spksrc == 'ch'):
    ch = myfile['Channels'].value
    loc = np.array([])
    nbins = 64
  elif (spksrc == 'cl'):
    locspks2d = myfile['Locations'].value
    ch = myfile['Cluster']['ClusterId'].value
    loc2d = myfile['Cluster']['CLocations'].value    
    loc = myfile['Cluster']['CLocationsS'].value  
    nbins = myfile['Cluster']['nBins'].value #768 #
    # removing clusters with id equal to -1
#    pdb.set_trace()
    valididxs = np.where(ch>=0)
    spikes = spikes[valididxs]
    amps = amps[valididxs]
    ch = ch[valididxs]    
    locspks2d = locspks2d[valididxs]    
       
  elif (spksrc == 'su'):
    ch = myfile['SortedUnits'].value
    markedch = myfile['MarkedChannels'].value
    valididxs = np.where(markedch>-1)
    spikes = spikes[valididxs]
    amps = amps[valididxs]
    ch = ch[valididxs]    
    loc = myfile['LocationsS'].value
    nbins = 64  
  data = {'spikes':spikes, 'amps': amps, 'ch':ch, 'freq':freq, 'nbins': nbins, 'loc':loc, 'loc2d':loc2d, 'locspks2d': locspks2d}
  return data


In [21]:
data = read_hdf5(datafullfn, spksrc=spksrc)
data

{'amps': array([ 1.14476037,  1.53721869,  1.53019202, ...,  2.91496754,
         1.83250916,  0.44643891], dtype=float32),
 'ch': array([214,   3, 120, ...,  33, 851,  32]),
 'freq': 7022.0,
 'loc': array([ 656,  289, 3755, ...,    0, 1007,    0]),
 'loc2d': array([[  9.99722195,  16.99970245],
        [  3.99506378,  34.0018692 ],
        [ 58.1168251 ,  35.79921722],
        ..., 
        [  0.        ,   0.        ],
        [ 15.6540947 ,   5.82002354],
        [ -0.10717568,   7.24974203]], dtype=float32),
 'locspks2d': array([[  4.99200249,  54.00999069],
        [ 42.89151764,  42.13150787],
        [ 41.17334747,  40.12137604],
        ..., 
        [ 55.31795502,  54.15864563],
        [ 47.5745697 ,   5.91076612],
        [ 39.00831604,  55.05916595]], dtype=float32),
 'nbins': 64,
 'spikes': array([      18,       20,       31, ..., 10290297, 10290302, 10290303])}

### Reading stimulus file

In [22]:
def read_stimfile(fullfn):
  # fullfn - string; a mat file containing the stimulus file whose variables are:
  # SamplingFrequency - float; sampling frequency of data acquisition, e.g. 7022
  # frameTimes - horizontal vector; times at which stimuli frames were presented, e.g. in a 15min white noise presentation with 100msec duration, 9000 images are presented, thus frameTimes is a <1x9000 double> vector  
  # frameSamples - horizontal vector; instead of time, it contains frame numbers at which stimuli were presented, i.e. frameSamples = frameTimes * SamplingFrequency 
  # pixels_allframes - 2D array; each row represents the pixels of the image presented at each frame, e.g. in a 15min white noise presentation with 100msec duration, 9000 images are presented, and if each image has 27*27 (=729) pixels, then pixels_allframes is a <9000x729 double> array  
  myfile = scipy.io.loadmat(fullfn)
  stimframes = np.array(myfile.get('frameSamples')[0], dtype='int')
  stimtimes = np.array(myfile.get('frameTimes')[0])
  stimpixels = np.array(myfile.get('pixels_allframes'), dtype='int')
  stimfreq = np.array(myfile.get('SamplingFrequency')[0][0])
  stim = {'stimframes':stimframes, 'stimtimes': stimtimes, 'stimfreq':stimfreq, 'stimpixels':stimpixels}  
  return stim

In [23]:
stim = read_stimfile(stimfullfn) 
stim

{'stimframes': array([1822238, 1836278, 1850319, 1864360, 1878400, 1892441, 1906481,
        1920522, 1934562, 1948603, 1962643, 1976684, 1990725, 2004765,
        2018805, 2032846, 2046887, 2060928, 2074968, 2089008, 2103049,
        2117089, 2131130, 2145170, 2159211, 2173252, 2187292, 2201332,
        2215373, 2229414, 2243455, 2257495, 2271535, 2285576, 2299617,
        2313657, 2327697, 2341738, 2355779, 2369819, 2383859, 2397900,
        2411941, 2425982, 2440022, 2454062, 2468103, 2482144, 2496184,
        2510224, 2524265, 2538306, 2552347, 2566386, 2580427, 2594468,
        2608508, 2622549, 2636589, 2650630]),
 'stimfreq': array(7022, dtype=uint16),
 'stimpixels': array([[255, 255, 255, ..., 255, 255, 255],
        [  0,   0,   0, ...,   0,   0,   0],
        [255, 255, 255, ..., 255, 255, 255],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [255, 255, 255, ..., 255, 255, 255],
        [  0,   0,   0, ...,   0,   0,   0]]),
 'stimtimes': array([ 259.504129

## Defining handy functions that will be called in later steps

get_spkidx(data, minamp, minframe, maxframe) returns indices of spikes, with minimum amplitude of minamp, which are fired between minframe and maxframe

In [24]:
def get_spkidx(data, minamp=7, minframe=0, maxframe=1000):
  amps = data['amps']
  spikes = data['spikes']
  idxframe_min = np.where(spikes >= minframe)[0]
  elmidx_beforemaxf = next(x[0] for x in enumerate(idxframe_min) if spikes[x[1]] > maxframe)
  idxframe = idxframe_min[:elmidx_beforemaxf]
  idxamp = np.where(amps >= minamp)[0]
  idx = np.intersect1d(idxframe, idxamp)
  return idx

get_hist_spkcnt(data, spkidx) returns the histogram of spikes whose indices are spkidx

In [25]:
def get_hist_spkcnt(data, spkidx):
  ch = data['ch']
  no_ch=max(ch[spkidx]) + 1
  hist_spkcnt = np.histogram(ch[spkidx],bins=np.arange(no_ch+1))[0]
  return hist_spkcnt

get_chlocation(ich, spksrc, nbins, loc, coord_spksrc) returns the location (locr, locc) of channel ich

In [26]:
def get_chlocation(ich, spksrc, nbins, loc, coord_spksrc='rc'):
  if 'array' in type(loc[ich]).__name__:
    locr = loc[ich][0]
    locc = loc[ich][1]
  elif spksrc == 'ch':
    locr = ich / nbins
    locc = np.mod(ich, nbins)
  elif spksrc == 'cl' or spksrc == 'su':
    if nbins:
      locr = loc[ich] / nbins
      locc = np.mod(loc[ich], nbins)
      locr = locr * 64 / nbins
      locc = locc * 64 / nbins  
    else:
      locr = loc[ich][0]
      locc = loc[ich][1]
  if coord_spksrc == 'cr':
    tmp = locr
    locr = locc
    locc = tmp
  return locr, locc

polish_fig(ax) polishes the figure passed by its axis, ax

In [27]:
def polish_fig(ax,xlabcoord=-.12,vis_rightaxis=False, vis_leftaxis=True, vis_topaxis=False, vis_bottomaxis=True, xtick_pos='bottom', ytick_pos='left', xlabel_pos='bottom', ylabel_pos='left', boldvline=True, invis_firstlabel=False, a=2):
  mplt.rcParams['xtick.direction'] = 'out'
  mplt.rcParams['ytick.direction'] = 'out'
  mplt.rc('axes',facecolor='ffffff')
  mplt.rc('axes',edgecolor='000000')
  mplt.rc('axes',labelcolor='000000')
  mplt.rc('xtick',color='000000')
  mplt.rc('ytick',color='000000')
  ax.spines["right"].set_visible(vis_rightaxis)
  ax.spines["top"].set_visible(vis_topaxis)
  ax.spines["left"].set_visible(vis_leftaxis)
  ax.spines["bottom"].set_visible(vis_bottomaxis)
  ax.xaxis.set_ticks_position(xtick_pos)
  ax.yaxis.set_ticks_position(ytick_pos)
  ax.xaxis.set_label_position(xlabel_pos)
  ax.yaxis.set_label_position(ylabel_pos)
  ax.xaxis.set_label_coords(0.5, xlabcoord)
      
  if invis_firstlabel:
    plt.setp(ax.get_yticklabels()[0], visible=False)    
    plt.setp(ax.get_xticklabels()[0], visible=False)    

#  ax.yaxis.set_major_locator(MaxNLocator(prune='lower'))
#  ax.tick_params(axis='x', pad=15) # distant ticks
  if boldvline:
    ax.axvline(linewidth=a, color="black")
    ax.axhline(linewidth=a, color="black")
    ax.tick_params('both', width=a, which='major', direction='out')


In [28]:
def plt_setlabels(titl='Response Latencies', xlab='Latency', ylab='Frequency', fntsz=16, xlabcoord=-.12, polishFLG=True):
    plt.title(titl,fontsize=fntsz)    
    plt.xlabel(xlab, fontsize=fntsz)
    plt.ylabel(ylab, fontsize=fntsz)
    if polishFLG:    
      ax = plt.gca()
      polish_fig(ax,xlabcoord=xlabcoord)

## Measuring neural responses to Full-field stimuli

In [29]:
# reading data and setting parameters

ch = data['ch']
spikes = data['spikes']
freq = data['freq']
nbins = data['nbins']
loc = data['loc']
loc2d = data['loc2d']
stimframes = stim['stimframes']  
no_frames = np.size(stimframes)
no_trials = no_frames/no_stim_pertrial
print 'no of trials:', no_trials

no of trials: 30


In [30]:
# determining spiking channels
spkidx = get_spkidx(data, minamp, stimframes[0], stimframes[-1])
hist_spkcnt = get_hist_spkcnt(data, spkidx)
allspikingChannels = np.where(hist_spkcnt >= minspk)[0] 
dur = (stimframes[1]-stimframes[0])/float(freq)

# the first trial is called as dark trial and the second trial as light trial, regardless of whether the dark was first presented or the light  
darktrials = zip(stimframes[0::2], stimframes[1::2])
stimframes_corc = np.append(stimframes, stimframes[-1]+ (dur*freq)) 
lighttrials = zip(stimframes[1::2], stimframes_corc[2::2])

if slctdchs == 'all':  
    spikingChannels = allspikingChannels
else:
    spikingChannels = slctdchs[np.where(hist_spkcnt[slctdchs]>=minspk)[0]]
print 'spiking channels:', spikingChannels

spiking channels: [6 7]


In [31]:
# initialization
spkcnt2d_avg, spkcnt2d_std, spkcnt2l_avg, spkcnt2l_std = (np.zeros(max(spikingChannels)+1) for i in range(4))
firstspk2d_avg, firstspk2d_std, firstspk2l_avg, firstspk2l_std = (np.zeros(max(spikingChannels)+1) for i in range(4))
latency2dark, latency2light, duresp2dark, duresp2light, peakresp2dark, peakresp2light = (np.zeros(max(spikingChannels)+1) for i in range(6))
biasidxs = np.ones(max(spikingChannels)+1) * -1.2
latency2dark_mtx, latency2light_mtx, duresp2dark_mtx, duresp2light_mtx = (np.zeros((64, 64)) for i in range(4))
latency2dark_mtx_no, latency2light_mtx_no, duresp2dark_mtx_no, duresp2light_mtx_no = (np.zeros((64, 64)) for i in range(4))
biasidx_mtx = np.zeros((64, 64))
biasidx_mtx_no = np.zeros((64,64))
locs_spikingChs = np.zeros((max(spikingChannels)+1,2))

This long piece of code computes and plots firing rate activity of all channels of multi-electrode array, including PSTH and raster plots 

In [32]:
  for ich in spikingChannels:
#    pdb.set_trace()
    print ich
    if spksrc == 'cl':
      locr, locc = get_chlocation(ich, spksrc, nbins, loc2d, coord_spksrc) 
    else:
      locr, locc = get_chlocation(ich, spksrc, nbins, loc, coord_spksrc) 
    #TEMP: change these back. I had to do it for P38_ret1
#    locr -= 1
#    locc -= 1
#    locs_spikingChs[ich] = int(locr*64+locc)
    locs_spikingChs[ich] = np.array([locr, locc])
    
    spkidx_ich = np.where(ch==ich)[0]
    spkidx_ich_ = np.intersect1d(spkidx_ich, spkidx)
    spkframe_ich = spikes[spkidx_ich_]
    
    # initialization
    spikes2dark_ich, spikes2light_ich = (np.array([]), np.array([]))
    firstspk2dark_ich, firstspk2light_ich = ([],[])
    spkcnt2dark_ich, spkcnt2light_ich = ([],[])
    
    # PSTH ******************************************************************    
    for itrial in range(no_trials):
        darktrial = darktrials[itrial]
        lighttrial = lighttrials[itrial]
        spkframe_darktrial = (spkframe_ich[(spkframe_ich>darktrial[0]) & (spkframe_ich<darktrial[1])]) - darktrial[0]
        spkframe_lighttrial = (spkframe_ich[(spkframe_ich>lighttrial[0]) & (spkframe_ich<lighttrial[1])]) - lighttrial[0]
        tdark = spkframe_darktrial/float(freq)
        tlight = spkframe_lighttrial/float(freq)
        spikes2dark_ich = np.append(spikes2dark_ich, tdark)
        spikes2light_ich = np.append(spikes2light_ich, tlight)
        if tdark.any(): 
          spkcnt2dark_ich.append(len(tdark))
          firstspk2dark_ich.append(min(tdark))
        if tlight.any():
          spkcnt2light_ich.append(len(tlight))                
          firstspk2light_ich.append(min(tlight))
    
    hist2dark, bin_edges_dark = np.histogram(spikes2dark_ich, bins=dur/dt, range=[0,dur])    
    hist2light, bin_edges_light = np.histogram(spikes2light_ich, bins=dur/dt, range=[0,dur])    
    hist2dark = hist2dark / float(no_trials*dt)
    hist2light = hist2light / float(no_trials*dt)
      
    hist2dl = np.append(hist2dark, hist2light)
    bin_edges_light = [b+dur for b in bin_edges_light]
    bin_edges = np.append(bin_edges_dark[:-1], bin_edges_light)
#    pdb.set_trace()
   
    # firing rate ***********************************************************    
    histdt = hist2dl * dt
    histdt_dark = hist2dark * dt
    histdt_light = hist2light * dt
    t=np.arange(-100,100,dt)
    X_plot = bin_edges[:,np.newaxis] #np.linspace(0,4,1000)[:,np.newaxis]
    if kernel == 'gaussian':
      kernelfnc = 1./(np.sqrt(2*np.pi)*dt) * np.exp(-np.power(t/dt,2)/2.0)  
    elif kernel == 'exponential':
      kernelfnc = 1./(np.sqrt(2)*dt) * np.exp(-np.sqrt(2) * np.abs(t/dt))  
#    pdb.set_trace()
    if convwithspktrainFLG:
      spktrain = np.append(spikes2dark_ich,spikes2light_ich+dur)
      spktrain.sort()
      # to estimate the best bandwidth (the estimate is much larger than what we think must be)
#      grid = GridSearchCV(KernelDensity(kernel='gaussian'),
#                    {'bandwidth': np.linspace(0.1, 1.0, 30)},
#                    cv=20) # 20-fold cross-validation
#      grid.fit(spktrain[:,np.newaxis])
#      print grid.best_params_
#      kde = grid.best_estimator_
      kde = KernelDensity(kernel=kernel, bandwidth=dt).fit(spktrain[:,np.newaxis])
      kde_logdens = kde.score_samples(X_plot)  
      rate = np.exp(kde_logdens)
      rate = rate * len(spktrain)/float(no_trials)
      rate2dark = rate[:len(rate)/2]
      rate2light = rate[len(rate)/2:-1]
    else:
      rate = np.convolve(kernelfnc, histdt) 
      rate2dark = np.convolve(kernelfnc, histdt_dark) 
      rate2light = np.convolve(kernelfnc, histdt_light) 
      rate = rate[np.floor(len(kernelfnc)/2)-1:len(rate)-np.ceil(len(kernelfnc)/2)+1]
      rate2dark = rate2dark[np.floor(len(kernelfnc)/2)-1:len(rate2dark)-np.ceil(len(kernelfnc)/2)+1]
      rate2light = rate2light[np.floor(len(kernelfnc)/2)-1:len(rate2light)-np.ceil(len(kernelfnc)/2)+1]
#    pdb.set_trace()
    tm = bin_edges 
    tmdark = bin_edges_dark
    tmlight = bin_edges_light

    # baseline *** ***********************************************************    
    #calculating baseline from spontaneous activity preceding the experiment 
    if baseline_dur<1:
      baseline = sum(hist2dark[-baseline_dur/dt:] + hist2light[-baseline_dur/dt:]) / np.float(2*baseline_dur/dt)
    else:  
      spks_ich_spont = np.where(spontdata['ch']==ich)[0] #spkidx_ich_spont=np.intersect1d(spkidx_ich, spkidx_spont)    
      baseline = len(spks_ich_spont) / float(starttime_exp)
#    pdb.set_trace()

#      idxframe_min = np.where(spikes >= minframe)[0]
#      elmidx_beforemaxf = next(x[0] for x in enumerate(idxframe_min) if spikes[x[1]] > maxframe)
#      idxframe = idxframe_min[:elmidx_beforemaxf]
#      idxamp = np.where(amps >= minamp)[0]
#      #  idxframe_max = np.where(spikes <= maxframe)[0]
#      #  idxframe = np.intersect1d(idxframe_min, idxframe_max)
  
#    print 'baseline spont:', baseline, '\nbaseline 250ms:', baseline2, '\n'    
#   Checking the zscore of the peaks *** **************************************
    p_peak_d = 1 - poisson.cdf(max(rate2dark), baseline+.00001) #the small amount added to the basline is for the problematic cases of having baseline=0
    p_peak_l = 1 - poisson.cdf(max(rate2light), baseline+.00001)
#    print 'baseline: ', baseline
#    print '\ndark peak:', max(rate2dark)
#    print 'Prob:', str(p_peak_d)
#    print '\nlight peak:', max(rate2light)
#    print 'Prob:', str(p_peak_l)
    
    if (checkRatePeakZFLG and (p_peak_d > max_poissonval_acceptable) and (p_peak_l > max_poissonval_acceptable)):
      print 'Rejected unit: peak firing rate too small\n\n'
      continue
     
#   PLotting firing rate ***********************************************************         
#    pdb.set_trace()    
    if frplotFLG:
#      fntsz=16
      plt.figure() 
#      pdb.set_trace()
      plt.bar(bin_edges[:-1], hist2dl, width=dt, color='grey')      
#      plt.plot(X_plot, rate_sklearn, 'r', lw=2)
      plt.plot(tm, rate, 'r',lw=2)
      plt_setlabels(titl=spksrc + ' ' + str(ich) + ' - loc (' + "%0.1f" % locr + ',' + "%0.1f" % locc + ')' + ' - spkcnt ' + str(hist_spkcnt[ich]), xlab='Time (sec)', ylab='Firing rate (Hz)', fntsz=18,xlabcoord=-.08)
#      plt_setlabels(titl='', xlab='Time (sec)', ylab='Firing rate (Hz)', fntsz=18,xlabcoord=-.08)
      plt.xlim([0,2*dur+.01])
      plt.axhline(y=baseline, c='grey', ls='--', lw=1)      
#      plt.text(0,0, '*', color='blue', fontsize=fntsz+2)
#      plt.text(dur,0, '*', color='green', fontsize=fntsz+2)
      plt.savefig(figpath+'rate_'+ spksrc +str(ich)+'_spkcnt'+str(hist_spkcnt[ich])+'_dt'+str(dt)+figformat)
      plt.close()
#   Cutting the rate function at the baseline *********************************
    rate2dark = np.array(rate2dark) - baseline
    rate2dark = [(r if r>0 else 0) for r in rate2dark]  
    rate2light = np.array(rate2light) - baseline
    rate2light = [(r if r>0 else 0) for r in rate2light]
    
#    pdb.set_trace()

    # Latency response ********************************************************   
    latencyd = 0.
    latencyl = 0.
    if ((p_peak_d < max_poissonval_acceptable) or (p_peak_l < max_poissonval_acceptable)): #mxrate
      mxrate = max(rate2dark)
      if mxrate:
        peakrateidx = np.where(rate2dark==mxrate)[0][0]
        latencyd = tmdark[peakrateidx] * 1000
        if (latencyd < maxlatency and latencyd > minlatency):
          latency2dark[ich] = latencyd
          peakresp2dark[ich] = mxrate
          latency2dark_mtx[round(locr), round(locc)] += latencyd
          latency2dark_mtx_no[round(locr), round(locc)] += 1.
  #          pdb.set_trace()

#    if (p_peak_l < max_poissonval_acceptable): #mxrate:    
      mxrate = max(rate2light)
      if mxrate:
        peakrateidx = np.where(rate2light==mxrate)[0][0]
        latencyl = (tmlight[peakrateidx] - dur) * 1000
        if (latencyl < maxlatency and latencyl > minlatency):
          latency2light[ich] = latencyl
          peakresp2light[ich] = mxrate
          latency2light_mtx[round(locr), round(locc)] += latencyl
          latency2light_mtx_no[round(locr), round(locc)] += 1.
  #          pdb.set_trace()

    if ((latencyd >= maxlatency or latencyd <= minlatency) and (latencyl >= maxlatency or latencyl <= minlatency)):
      print 'Rejected unit: latency is too high or too low; latencyd', latencyd, 'latencyl', latencyl, '\n\n'
      latency2dark[ich], latency2light[ich], peakresp2dark[ich], peakresp2light[ich] = (0,0,0,0)
      continue
#      print '\nlatency:', latencyd, latencyl
#      pdb.set_trace()
    # Duration of response ****************************************************
#    pdb.set_trace()
    durespd = 0
    durespl = 0
    #if (p_peak_d < max_poissonval_acceptable): 
    if latency2dark[ich]:
      durespd = dur * 1000
      mxrate = max(rate2dark)
      mxrate_tau = mxrate / np.e
      peakrateidx = np.where(rate2dark==mxrate)[0][0]
      for i in range(peakrateidx+1, len(rate2dark)):
        if rate2dark[i] < mxrate_tau:
          # estimating the exact timing f mxrate_tau: (y-y1)(x2-x1)/y2-y1 + x1
          durespd =  (mxrate_tau - rate2dark[i-1])*(tmdark[i] - tmdark[i-1])/(rate2dark[i] - rate2dark[i-1]) + tmdark[i-1]
          durespd = (durespd - tmdark[peakrateidx]) * 1000
          break
      if durespd < maxduration:
#              print 'low duration d: ch ', ich
        duresp2dark[ich] = durespd
        duresp2dark_mtx[round(locr), round(locc)] += durespd
        duresp2dark_mtx_no[round(locr), round(locc)] += 1.

    #if (p_peak_l < max_poissonval_acceptable): #mxrate: 
    if latency2light[ich]:
      durespl = dur * 1000
      mxrate = max(rate2light)
      mxrate_tau = mxrate / np.e
      peakrateidx = np.where(rate2light==mxrate)[0][0]
      for i in range(peakrateidx+1, len(rate2light)):
        if rate2light[i] < mxrate_tau:
          # estimating the exact timing f mxrate_tau: (y-y1)(x2-x1)/y2-y1 + x1
          durespl =  (mxrate_tau - rate2light[i-1])*(tmlight[i] - tmlight[i-1])/(rate2light[i] - rate2light[i-1]) + tmlight[i-1]
          durespl = (durespl - tmlight[peakrateidx]) * 1000            
          break
      if durespl < maxduration:
        duresp2light[ich] = durespl
        duresp2light_mtx[round(locr), round(locc)] += durespl
        duresp2light_mtx_no[round(locr), round(locc)] += 1.
      
    if (durespd > maxduration and durespl > maxduration):
      print 'Rejected unit: duration is too high; durespd', durespd, 'durespl', durespl, '\n\n'
      latency2dark[ich], latency2light[ich], peakresp2dark[ich], peakresp2light[ich] = (0,0,0,0)
      duresp2dark[ich], duresp2light[ich] = (0,0)
      continue
      
#      print '\nresponse duration:', durespd, durespl
#    pdb.set_trace()      
    # Biasindex response ******************************************************    
    if ((p_peak_d < max_poissonval_acceptable) or (p_peak_l < max_poissonval_acceptable)): #(mxrate2dark or mxrate2light):
      mxrate2dark = 0
      mxrate2light = 0
#      if (p_peak_d < max_poissonval_acceptable):
      if latency2dark[ich]:
        mxrate2dark = max(rate2dark)
#      if (p_peak_l < max_poissonval_acceptable):
      if latency2light[ich]:
        mxrate2light = max(rate2light)
      #print latency2dark[ich], latency2light[ich]
      biasidx = (mxrate2light - mxrate2dark) / float(mxrate2dark + mxrate2light) 
      if firststim == 'l':
        biasidx *= -1
      biasidxs[ich] = biasidx
      biasidx_mtx[round(locr), round(locc)] += biasidx
      biasidx_mtx_no[round(locr), round(locc)] += 1.
      print 'bias index:', biasidx, '\n'
#      pdb.set_trace()
    
    if d2lcmpFLG and (not biasidxFLG):
      raise ValueError('Please set biasidxFLG to True!')

#   Setting spike counts and first spike measures ******************************     
    if len(spkcnt2dark_ich):
      spkcnt2d_avg[ich]=np.mean(spkcnt2dark_ich)
      spkcnt2d_std[ich]=np.std(spkcnt2dark_ich)
      firstspk2d_avg[ich]=np.mean(firstspk2dark_ich)
      firstspk2d_std[ich]=np.std(firstspk2dark_ich)
      
    if len(spkcnt2light_ich):          
      spkcnt2l_avg[ich]=np.mean(spkcnt2light_ich)
      spkcnt2l_std[ich]=np.std(spkcnt2light_ich)
      firstspk2l_avg[ich]=np.mean(firstspk2light_ich)
      firstspk2l_std[ich]=np.std(firstspk2light_ich)

6
bias index: 1.0 

7
bias index: 1.0 



## Saving figures and computed parameters

In [33]:
  print 'figs were saved in ', figpath
  # ************************************PLOTS *********************************
  ylab='channels'
  if spksrc=='cl':
    ylab = 'clusters'
  elif spksrc == 'su':
    ylab = 'units'

  if firststim == 'l':
    latency2dark, latency2light = latency2light, latency2dark
    latency2dark_mtx, latency2light_mtx = latency2light_mtx, latency2dark_mtx
    duresp2dark, duresp2light = duresp2light, duresp2dark
    duresp2dark_mtx, duresp2light_mtx = duresp2light_mtx, duresp2dark_mtx
    firstspk2d_avg, firstspk2l_avg = firstspk2l_avg, firstspk2d_avg      
    firstspk2d_std, firstspk2l_std = firstspk2l_std, firstspk2d_std
    spkcnt2d_avg, spkcnt2l_avg = spkcnt2l_avg, spkcnt2d_avg
    spkcnt2d_std, spkcnt2l_std = spkcnt2l_std, spkcnt2d_std
    peakresp2dark, peakresp2light = peakresp2light, peakresp2dark
#  pdb.set_trace()
  if frsaveparamsFLG:
    print 'saving fr params in a mat file...'
    frparamnames = np.array(['Spikingchannels', 'locs_r', 'locs_c','biasidxs', 'latency2l', 'latency2d',\
              'duresp2l', 'duresp2d', 'peakresp2l', 'peakresp2d', 'firstspk2l_avg', 'firstspk2d_avg', \
              'firstspk2l_std', 'firstspk2d_std', 'spkcnt2l_avg', 'spkcnt2d_avg', 'spkcnt2l_std', 'spkcnt2d_std'])
    frparams = np.transpose(np.vstack([spikingChannels, locs_spikingChs[spikingChannels, 0], locs_spikingChs[spikingChannels, 1], biasidxs[spikingChannels], latency2light[spikingChannels], latency2dark[spikingChannels],\
              duresp2light[spikingChannels], duresp2dark[spikingChannels], peakresp2light[spikingChannels], peakresp2dark[spikingChannels], firstspk2l_avg[spikingChannels], firstspk2d_avg[spikingChannels], \
              firstspk2l_std[spikingChannels], firstspk2d_std[spikingChannels], spkcnt2l_avg[spikingChannels], spkcnt2d_avg[spikingChannels], spkcnt2l_std[spikingChannels], spkcnt2d_std[spikingChannels]]))
    frparams = frparams.astype(object)
    frparams = np.insert(frparams, 0, frparamnames, axis=0)
    frparamsf = frparams_matfile+'_'+str(len(spikingChannels))+spksrc+'.mat'
    scipy.io.savemat(frparamsf,mdict={'FRparams':frparams}, oned_as='column')
    print '\nParameters were saved into: ', frparamsf

figs were saved in  /Users/sahar/Documents/spktanlz/data/retina/P38_06Mar14/ret2/figs/
saving fr params in a mat file...

Parameters were saved into:  /Users/sahar/Documents/spktanlz/data/retina/P38_06Mar14/ret2/figs/frparams_2cl.mat
